In [1]:
import numpy as np
import pandas as pd
import os 
import glob
import pdb
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import scipy as sp
from scipy.special import expit
import sys

In [2]:
sys.path.append(os.path.dirname(os.getcwd()))
from visualisation_and_evaluation.helpers_eval import extract_scores, wa, select_best_run
from visualisation_and_evaluation.helpers_vizualisation import plot_scores

In [3]:
path_scores = '/Users/joannaf/Desktop/courses/DeepLearning/DL2019/project/data/Dataset5/'

files = [x for x in os.listdir(path_scores) if 'full' in x and 'score' in x]
selection_method = 'wa' #'wa' (weighted average), 'div' (based on divergence score only)
eval_all = pd.read_csv(path_scores+'scores_baselines_full.csv', index_col=[0])
eval_all['silhouette_score_neg'] = -eval_all['silhouette_score']
for fname in files:
    if(fname == 'scores_baselines_full.csv'):
        continue
    df = extract_scores(path_scores, fname)
    # select "the best run"
    df_best = select_best_run(df, method=selection_method)
    df_best = df_best.loc[:,eval_all.columns]
    eval_all = eval_all.append(df_best)
eval_all = eval_all.sort_values(['sample', 'method']).reset_index(drop=True)


/Users/joannaf/Desktop/courses/DeepLearning/DL2019/project/2019_DL_Class/visualisation_and_evaluation/helpers_eval.py:205: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.index_name = ['epoch']


In [4]:
# add the weighted averaged score
eval_all['wa'] = eval_all[['divergence_score', 'entropy_score', 'silhouette_score_neg']].apply(lambda x: wa(x), axis=1)
eval_all


,sample,method,divergence_score,entropy_score,silhouette_score,silhouette_score_neg,wa
0,sample5,autoencoder gan bottleneck batchnorm,0.162493,0.668361,0.056609,-0.056609,0.565778
1,sample5,autoencoder gan diamond batchnorm,0.089144,0.677833,0.067794,-0.067794,0.556724
2,sample5,combat,0.365523,0.562519,0.199534,-0.199534,0.576355
3,sample5,mnn,0.872710,0.608623,0.207280,-0.207280,0.636616
4,sample5,raw,1.616501,0.431192,0.188119,-0.188119,0.689625
5,sample5,reg,2.004377,0.154429,0.197928,-0.197928,0.692323
6,sample5,vanilla gan batchnorm,0.211468,0.638272,-0.205391,0.205391,0.582878
7,sample65,combat,0.356872,0.534817,0.227501,-0.227501,0.571997
8,sample65,mnn,0.954991,0.612436,0.110817,-0.110817,0.650073
9,sample65,raw,2.037434,0.476793,0.155715,-0.155715,0.719663


In [ ]:
### plot scores
plot_scores(eval_all, xcol='divergence_score', ycol='entropy_score')
plot_scores(eval_all, xcol='divergence_score', ycol='silhouette_score_neg')

eval_sample5 = eval_all.loc[eval_all['sample']=='sample5',:]
title5 = 'Evaluation scores: sample5'
plot_scores(eval_sample5, xcol='divergence_score', ycol='entropy_score', title= title5)
plot_scores(eval_sample5, xcol='divergence_score', ycol='silhouette_score_neg', title=title5)